## Advanced RAG - Retrieval Strategies
### Dense, Sparse, Hybrid, and Reranking

**Objectives:**
- Understand different retrieval modes
- Implement hybrid search with filters
- Apply reranking for better results
- Build reusable retrieval functions

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse

# re-ranking for better result
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# metadata filtering
from qdrant_client.models import Filter, FieldCondition, MatchValue

# metadata extraction from LLM
from scripts.schema import ChunkMetadata